In [157]:
import pandas as pd
import numpy as np
import ast
import warnings
warnings.filterwarnings('ignore')

In [158]:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [159]:
catalog_df = pd.read_csv('drive/MyDrive/collab_files/product_catalog.csv')
catalog_df

,product_id,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]"
1,28640,1366,10,1,537,0,101,NaN
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3..."
3,21399,1090,10,1,511,0,0,[3270]
4,8504,768,4,1,484,0,66,[2470]
...,...,...,...,...,...,...,...,...
32771,12036,1383,10,0,503,0,101,"[3502, 1088, 2004, 691, 3422, 2308, 308, 3688,..."
32772,22709,567,4,2,491,3,66,"[3900, 3905, 3903, 3910, 1807, 274, 1467, 914]"
32773,32322,1385,10,1,500,0,37,"[2686, 2609, 725]"
32774,19118,1113,6,2,491,0,117,"[3900, 3910, 3903, 2473, 914, 274]"


In [160]:
print("# -1 values in attributes")
for i in range(1,6):
  str1 = 'attribute_'+ str(i)
  print(f"{str1} : {catalog_df[catalog_df[str1]==-1]['product_id'].count()}")

# -1 values in attributes
attribute_1 : 4
attribute_2 : 0
attribute_3 : 466
attribute_4 : 617
attribute_5 : 4264


In [161]:
for i in range(1,5):
  str1 = 'attribute_'+ str(i)
  catalog_df[str1] = catalog_df[str1].replace(-1, catalog_df[str1].mode()[0])

In [162]:
pd.set_option('display.max_rows', None)       # Show all rows
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [163]:
# Step 1: Calculate the mode for each group
mode_values = catalog_df.groupby(['attribute_1', 'attribute_2','attribute_3'])['categories'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else None)

# Step 2: Map the mode values back to the original DataFrame
catalog_df['mode_fill'] = catalog_df.set_index(['attribute_1', 'attribute_2','attribute_3']).index.map(mode_values)

# Step 3: Fill NaN values in 'categories' with the corresponding mode values
catalog_df['categories'].fillna(catalog_df['mode_fill'], inplace=True)

# Drop the temporary 'mode_fill' column if no longer needed
catalog_df.drop(columns=['mode_fill'], inplace=True)

In [164]:
catalog_df.isna().sum()

,0
product_id,0
manufacturer_id,0
attribute_1,0
attribute_2,0
attribute_3,0
attribute_4,0
attribute_5,0
categories,197


In [165]:
catalog_df['categories'].fillna(catalog_df['categories'].mode()[0],inplace=True)

In [166]:
catalog_df['categories'] = catalog_df['categories'].astype(str)

In [167]:
# catalog_df['categories'] = catalog_df['categories'].apply(ast.literal_eval)

In [168]:
# def assign_category(cat_list):
#     if len(cat_list) == 1 and isinstance(cat_list[0],int):
#         return int(cat_list[0])
#     for target in target_categories:
#         if target in cat_list:
#             return int(target)
#     return cat_list
# catalog_new = catalog_df.copy()
# catalog_new['categories'] = catalog_df['categories'].apply(assign_category)
# catalog_new.head(10)

In [169]:
# type(catalog_new.loc[32655,'categories'])

In [170]:
# # Filter rows where 'categories' column entries are not integers
# non_integer_rows = catalog_new[catalog_new['categories'].apply(lambda x: not isinstance(x, int))].reset_index(drop=True)
# len(non_integer_rows)

In [171]:
# from collections import Counter


# # String formatındaki listeleri dönüştürme
# # Tüm kategorileri bir araya getirme
# all_categories = [item for sublist in non_integer_rows['categories'] for item in sublist]

# # En çok tekrar eden kategorileri bulma
# category_counts = Counter(all_categories)
# most_common_categories = category_counts.most_common(20)

# # Sonucu yazdırma
# print(most_common_categories)


In [172]:
category_map_df = pd.read_csv("/content/drive/MyDrive/collab_files/product_category_map.csv")
category_map_df.head()

,category_id,parent_category_id
0,0,75
1,1,1499
2,2,1082
3,3,3498
4,4,1623


In [173]:
category_map_df = category_map_df.drop_duplicates(subset=['category_id'])
category_map_df[category_map_df['category_id'].duplicated(keep=False)]

,category_id,parent_category_id


In [174]:
category_map_df[category_map_df['category_id']==category_map_df['parent_category_id']]['category_id'].values

array([3898])

In [175]:
category_map_df.loc[category_map_df['category_id'] == 3908, 'parent_category_id']


,parent_category_id
3940,3898


In [176]:
def category_mapper(df):
    # Step 1: Identify top-level categories (where category_id equals parent_category_id)
    top_nodes = df[df['category_id'] == df['parent_category_id']]['category_id'].values

    # Step 2: Create a copy of the input DataFrame to avoid modifying the original
    category_map_df_filtered = df.copy()

    # Step 3: Update parent_category_id for categories that are children of top-level categories
    # For rows where parent_category_id is in the list of top-level categories,
    # set parent_category_id to be the same as category_id so they become top category
    category_map_df_filtered.loc[category_map_df_filtered['parent_category_id'].isin(top_nodes), 'parent_category_id'] = category_map_df_filtered['category_id']

    # Step 4: Find categories where category_id equals parent_category_id after the update
    # These are categories that are now "self-referencing" (i.e., a category that is its own parent)
    parent_list = category_map_df_filtered[category_map_df_filtered['category_id'] == category_map_df_filtered['parent_category_id']]['category_id'].values

    # Step 5: Filter the parent_list to include only those category_ids that are <= 3898
    # Heuristically categories above 3898 is not counted as parent
    parent_list = [x for x in parent_list if x <= 3898]

    # Step 6: Return the modified DataFrame and the filtered list of parent category IDs
    return category_map_df_filtered, parent_list



In [177]:
catalog_df['categories'] = catalog_df['categories'].apply(ast.literal_eval)
catalog_df['categories'] = catalog_df['categories'].apply(lambda x: sorted(x))
catalog_df.loc[0,'categories']

[855, 2890, 3908, 3909]

In [178]:
def category_row_transform(row, df_map, parents):
    category = row['categories']
    element_list = []
    for element in category:
      while element not in parents:
          element = df_map.loc[df_map['category_id'] == element, 'parent_category_id'].values[0]
      element_list.append(element)
    category = list(set(element_list))
    row['categories'] = category
    return row


In [179]:
category_map_df_filtered, parent_list = category_mapper(category_map_df)

In [180]:
category_map_df_filtered.loc[category_map_df_filtered['parent_category_id'] >= 3898, 'parent_category_id'] = 3898
category_map_df_filtered = category_map_df_filtered.drop_duplicates(subset=['category_id', 'parent_category_id'])

In [181]:
catalog_df_transformed = catalog_df.apply(lambda row: category_row_transform(row,category_map_df_filtered,parent_list),axis=1)

In [182]:
catalog_df_transformed.loc[0,'categories']

[3898, 2012, 2838]

In [183]:
catalog_df_transformed.head()

,product_id,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,categories
0,22665,861,4,0,490,2,66,"[3898, 2012, 2838]"
1,28640,1366,10,1,537,0,101,[34]
2,13526,1090,10,0,511,0,0,"[600, 3898, 3860, 2364]"
3,21399,1090,10,1,511,0,0,[2364]
4,8504,768,4,1,484,0,66,[1072]


In [184]:
from collections import Counter
def similar_categories(category,similar_count):
  filtered_categories = catalog_df_transformed[
      catalog_df_transformed['categories'].apply(lambda x: isinstance(x, list) and category in x)
  ]['categories']

  # Flatten the filtered lists into a single list
  all_elements = [item for sublist in filtered_categories for item in sublist]

  # Count occurrences of each element
  value_counts = Counter(all_elements)

  # Convert to a sorted DataFrame for better presentation
  value_counts_df = pd.DataFrame(value_counts.items(), columns=['Category', 'Count']).sort_values(by='Count', ascending=False)

  return value_counts_df.iloc[1:similar_count+1, 0].tolist()


In [185]:
similar_categories(1825,5)

[366, 1677, 3898, 3624, 2364]

In [186]:
transactions_df= pd.read_csv("drive/MyDrive/collab_files/transactions.csv")
transactions_df.head()

,customer_id,product_id,purchase_date,quantity
0,38769,3477,2020-06-01,1
1,42535,30474,2020-06-01,1
2,42535,15833,2020-06-01,1
3,42535,20131,2020-06-01,1
4,42535,4325,2020-06-01,1


In [187]:
catalog_df_transformed2 = transactions_df.merge(catalog_df_transformed, on='product_id', how='left')
catalog_df_transformed2.head()

,customer_id,product_id,purchase_date,quantity,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,categories
0,38769,3477,2020-06-01,1,186,6,0,196,0,45,"[3056, 3898, 1543]"
1,42535,30474,2020-06-01,1,193,10,3,229,3,132,"[1666, 621, 3056, 3898, 2364]"
2,42535,15833,2020-06-01,1,1318,4,1,455,0,108,"[3624, 252, 519]"
3,42535,20131,2020-06-01,1,347,4,0,291,3,44,"[1763, 3624, 2793, 2762, 621, 1682, 1557, 2231, 3898, 1565]"
4,42535,4325,2020-06-01,1,539,6,0,303,0,45,"[3624, 1682, 2231, 3898, 2364]"


In [188]:
import pandas as pd
from itertools import combinations
from collections import defaultdict
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Flatten transactions into category pairs
def build_cooccurrence_matrix(df):
    cooccurrence = defaultdict(int)

    # Iterate through each product's category list
    for categories in df['categories']:
        if isinstance(categories, list):
            # Generate all unique pairs of categories
            for pair in combinations(categories, 2):
                cooccurrence[tuple(sorted(pair))] += 1

    # Convert to DataFrame
    cooccurrence_df = pd.DataFrame(
        [(k[0], k[1], v) for k, v in cooccurrence.items()],
        columns=['Category1', 'Category2', 'Count']
    )
    return cooccurrence_df

# Build the co-occurrence matrix
cooccurrence_df = build_cooccurrence_matrix(catalog_df_transformed)

# Step 2: Convert co-occurrence into a similarity matrix
# Create a mapping of categories to indices
all_categories = set(cooccurrence_df['Category1']).union(cooccurrence_df['Category2'])
category_index = {category: i for i, category in enumerate(all_categories)}

# Initialize a square matrix of zeros
cooccurrence_matrix = np.zeros((len(all_categories), len(all_categories)))

# Fill in the matrix with co-occurrence counts
for _, row in cooccurrence_df.iterrows():
    i, j = category_index[row['Category1']], category_index[row['Category2']]
    cooccurrence_matrix[i, j] = row['Count']
    cooccurrence_matrix[j, i] = row['Count']  # Symmetric matrix


In [189]:
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Normalize the co-occurrence matrix (optional, depending on your data)
normalized_matrix = cooccurrence_matrix / np.max(cooccurrence_matrix)

# Step 2: Compute the similarity matrix
similarity_matrix = cosine_similarity(normalized_matrix)

# Step 3: Convert similarity to a distance matrix
distance_matrix = 1.000000001 - similarity_matrix  # DBSCAN requires distances

# Step 4: Apply DBSCAN
eps = 0.15  # Adjust epsilon based on your dataset
min_samples = 2  # Minimum samples for a core point
dbscan_model = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')

# Fit the model
cluster_labels = dbscan_model.fit_predict(distance_matrix)

In [190]:
# Map categories back to clusters
category_cluster_mapping = {category: cluster for category, cluster in zip(category_index.keys(), cluster_labels)}

# Step 4: Assign clusters to products
def assign_product_cluster(categories, cluster_mapping):
    product_clusters = [cluster_mapping[category] for category in categories if category in cluster_mapping]
    return max(set(product_clusters), key=product_clusters.count) if product_clusters else -1  # Assign most common cluster

catalog_df_transformed2['product_cluster'] = catalog_df_transformed2['categories'].apply(
    lambda x: assign_product_cluster(x, category_cluster_mapping)
)

# Step 5: View results
catalog_df_transformed2.drop(columns=['categories'],inplace=True)
catalog_df_transformed2.sort_values('product_id').tail(50)

,customer_id,product_id,purchase_date,quantity,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,product_cluster
775757,8537,32769,2020-12-02,1,1383,6,0,502,0,124,-1
650843,36240,32769,2020-11-06,4,1383,6,0,502,0,124,-1
938294,10158,32769,2021-01-07,1,1383,6,0,502,0,124,-1
884653,23235,32769,2020-12-26,2,1383,6,0,502,0,124,-1
931023,29558,32769,2021-01-05,1,1383,6,0,502,0,124,-1
372012,10583,32769,2020-08-28,3,1383,6,0,502,0,124,-1
396955,35730,32769,2020-09-02,1,1383,6,0,502,0,124,-1
936512,30993,32769,2021-01-06,1,1383,6,0,502,0,124,-1
6514,43406,32769,2020-06-02,1,1383,6,0,502,0,124,-1
708461,41375,32769,2020-11-20,1,1383,6,0,502,0,124,-1


In [191]:
catalog_df_transformed = catalog_df_transformed2
catalog_df_transformed.head()

,customer_id,product_id,purchase_date,quantity,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,product_cluster
0,38769,3477,2020-06-01,1,186,6,0,196,0,45,0
1,42535,30474,2020-06-01,1,193,10,3,229,3,132,-1
2,42535,15833,2020-06-01,1,1318,4,1,455,0,108,0
3,42535,20131,2020-06-01,1,347,4,0,291,3,44,0
4,42535,4325,2020-06-01,1,539,6,0,303,0,45,0


In [192]:
catalog_df_transformed.value_counts('product_cluster')

,count
product_cluster,
0,743399
-1,293287
3,27018
2,7790
1,44
